#PGP (Pretty Good Privacy)
PGP es una técnica de cifrado usada para enviar correos electrónicos seguros.
Su objetivo para un mensaje es que:

  * Nadie pueda leerlo excepto el destinatario (confidencialidad).

  * No pueda ser alterado sin que se detecte (integridad).

  * El destinatario pueda confirmar quién lo envió (autenticidad).

Para lograr todo esto, PGP usa criptografía de llave pública.

## Paso 1: Preparar el entorno

In [1]:
!pip install python-gnupg > /dev/null # python-gnupg: permite usar PGP desde Python.
!apt-get install -y gnupg > /dev/null # gnupg: es el programa que genera y maneja llaves.

import gnupg
import os
import shutil

print("Entorno preparado para la simulación PGP...\n")

Entorno preparado para la simulación PGP...



## Paso 2: Configuro la simulacíon
Simulo dos usuarios reales que van a enviarse un correo cifrado.

Creo dos carpetas: una para “Rosa” y otra para “Luis”.
Cada una funciona como si fuera un “computador” separado con su propio sistema de llaves.

In [5]:
# Definimos carpetas temporales en la nube para Rosa y Luis
base_path = "/content/pgp_lab"

if os.path.exists(base_path):
    shutil.rmtree(base_path)

rosa_home = os.path.join(base_path, "rosa")
luis_home = os.path.join(base_path, "luis")

os.makedirs(rosa_home)
os.makedirs(luis_home)

# Inicializamos las instancias de GPG apuntando a esas carpetas
gpg_rosa = gnupg.GPG(gnupghome=rosa_home, use_agent=False)
gpg_luis = gnupg.GPG(gnupghome=luis_home, use_agent=False)

##Paso 3: Protejo las contraseñas
Antes de generar las llaves, convierto las contraseñas de los usuarios en un hash usando el algoritmo SHA-256, para que no se muestren en texto plano.

In [13]:
import hashlib

def hash_pass(password: str) -> str:
    return hashlib.sha256(password.encode()).hexdigest()

# Hashear contraseñas
pass_rosa = hash_pass("password123")
pass_luis  = hash_pass("password456")

print("Las contraseñas han sido protegidas.")

Las contraseñas han sido protegidas.


## Paso 3: Generación de llaves
Aquí cada usuario crea su par de llaves:

* Llave pública: se envía a los demás.

* Llave privada: se guarda en secreto y lleva una contraseña.

Rosa y Luis tienen su identidad digital, igual que en cualquier sistema PGP.

In [14]:
print("Generando llaves (esto puede tardar unos segundos)...")

# Definimos los parámetros de la llave
key_input_rosa = gpg_rosa.gen_key_input(
    key_type="RSA",
    key_length=2048,
    name_real="Rosa",
    name_email="rosa@colab.com",
    passphrase=pass_rosa
)
key_rosa = gpg_rosa.gen_key(key_input_rosa)
print(f"   -> Llave de Rosa generada: {key_rosa.fingerprint[:8]}...")

key_input_luis = gpg_luis.gen_key_input(
    key_type="RSA",
    key_length=2048,
    name_real="Luis",
    name_email="luis@colab.com",
    passphrase=pass_luis
)
key_luis = gpg_luis.gen_key(key_input_luis)
print(f"   -> Llave de Luis generada:   {key_luis.fingerprint[:8]}...")

Generando llaves (esto puede tardar unos segundos)...
   -> Llave de Rosa generada: 9C6E0428...
   -> Llave de Luis generada:   976F8C62...


## PASO 4: Intercambio de llaves públicas
Rosa envía su llave pública a Luis, y Luis hace lo mismo.
Así pueden cifrar mensajes entre sí.

In [15]:
print("\nIntercambiando llaves públicas...")

# 1. Rosa exporta su pública -> Luis la importa
rosa_pub_key = gpg_rosa.export_keys(key_rosa.fingerprint)
import_result_luis = gpg_luis.import_keys(rosa_pub_key)
gpg_luis.trust_keys(import_result_luis.fingerprints, 'TRUST_ULTIMATE') # Luis confía ciegamente en Rosa

# 2. Luis exporta su pública -> Rosa la importa
luis_pub_key = gpg_luis.export_keys(key_luis.fingerprint)
import_result_rosa = gpg_rosa.import_keys(luis_pub_key)
gpg_rosa.trust_keys(import_result_rosa.fingerprints, 'TRUST_ULTIMATE') # Rosa confía ciegamente en Luis

print(" -> Llaves intercambiadas y firmadas.")


Intercambiando llaves públicas...
 -> Llaves intercambiadas y firmadas.


## PASO 5: Cifrado y firma del mensaje
Rosa hace dos cosas:

* Cifra el mensaje con la llave pública de Luis → solo Luis puede abrirlo.

* Firma el mensaje con su llave privada → Luis sabrá que realmente lo envió Rosa.

In [18]:
mensaje = "Luis, te envío este mensaje cifrado porque ya me cansé de que ese hacker lea mis chismes."

print(f"\nMensaje original: '{mensaje}'")

# Rosa CIFRA para Luis (usa llave pública de Luis) y FIRMA (usa su privada)
datos_cifrados = gpg_rosa.encrypt(
    mensaje,
    recipients=['luis@colab.com'],
    sign=key_rosa.fingerprint,
    passphrase=pass_rosa
)

print("\n🔒 MENSAJE CIFRADO (Lo que vería un hacker):")
print(str(datos_cifrados))


Mensaje original: 'Luis, te envío este mensaje cifrado porque ya me cansé de que ese hacker lea mis chismes.'

🔒 MENSAJE CIFRADO (Lo que vería un hacker):
-----BEGIN PGP MESSAGE-----

hQEMAwJPgZTCr9ZWAQgAgblbdKawl9y9Q8H/2s2I2Gvahy5S/h/j4ZNOSVMz+PaO
BbTxJtTKPWzC+CGtcAahVLiHLLCk7Xt20rEB+JMBT8sXAbXNcHGR2MBTbOisRcUw
sYX7IPYg0wDlAqeoiLin81h+Cc1Z0rMo7U32G4QRkNuiVuRuBNrBXpHLjkxZ1cIo
c7WNQXsJRV8HfL5FZI6az4iU780xeYmXVu7bM2K0vB2ZjWXSXy54XQ7j3Q+mzB0E
PcfBqtlN7hT2WRx9+DTeLuoU1SRn04hBJbVnmrZ6JhdxNPpsSz4Q9pxYum+9M2IW
chidgXWNB1Magr4ldUjhMtkmjem7x6YuKjrMsVpZtNLBFgG3NJS0JYtgHp8I8FrX
aCdCOmK2WxP2etmRndFR7Au+25caXFsSRPHZArEd8T/ICOY2OlgG1QFA9Pws9BX6
NROnGXG5G9baswjQg6eNQ8KQF/1c6vZFxBFU0ZTGHPPiB3k7zM0PURZXKvdmf0Om
wOajJW6HDaqNIgTZBN9hIzn3yyeorXeUrjM4yEIAu2cc5KaH2Y+LDt/yggC5NsOI
c8kkhq5tJI5EKD19u2y1jbGsuKQIXO+F9OUNUzaH53Adkq6mMJ62wE+IFQvhFXKS
nfS0Odnpc3kgW9MTdRqck6KMQ9A4CH2kJ5SVIejYflsWxqtsR1l/V5aZh1Aqqs+o
GdyCdd0mWettkspKfCXwIeuyoARAi7+ljbhyDvrWcjLW1UtR5o8GyniiJwXKTsFm
mgDUsFSugAGZQqUrpJT/RAhY+Em1fno53J9

## PASO 6: Decifrado y verificación del mensaje
Cuando Luis recibe el mensaje:

* Usa su llave privada + su contraseña para descifrar.

* El sistema verifica la firma para confirmar que lo envió Rosa.

In [19]:
print("\n🔓 Luis intentando descifrar el mensaje...")

# Luis descifra usando su contraseña privada (correcta)
datos_descifrados_acertado = gpg_luis.decrypt(
    str(datos_cifrados),
    #passphrase='password_incorrecto'
    passphrase=pass_luis #Una vez que ingresa la clave correcta esta queda guardada en cache
)

if datos_descifrados_acertado.ok:
    print(f"   ✅ Mensaje descifrado: '{datos_descifrados_acertado.data.decode('latin-1')}'")
    print(f"   ✅ Firma verificada: {datos_descifrados_acertado.valid}")
    print(f"   ✅ Firmado por: {datos_descifrados_acertado.username}")
else:
    print("   ❌ Error al descifrar")
    print(datos_descifrados_acertado.status)


🔓 Luis intentando descifrar el mensaje...
   ✅ Mensaje descifrado: 'Luis, te envío este mensaje cifrado porque ya me cansé de que ese hacker lea mis chismes.'
   ✅ Firma verificada: True
   ✅ Firmado por: Rosa <rosa@colab.com>


## Ejemplo de error al intentar descifrar un mensaje no cifrado.
Aquí Luis intenta descifrar un mensaje de texto plano.
Como no es un mensaje PGP, el sistema devuelve un error.

In [25]:
print("\n🔴 Luis intentando descifrar un mensaje NO CIFRADO (Ejemplo de Error)...")

mensaje_no_cifrado = "Este es un mensaje en texto plano, no cifrado."

# Luis intenta descifrar un mensaje que no es PGP cifrado
datos_descifrados_error = gpg_luis.decrypt(
    mensaje_no_cifrado,
    passphrase=pass_luis # La contraseña aquí es irrelevante para datos no PGP
)

if datos_descifrados_error.ok:
    print(datos_descifrados_error.data.decode('latin-1'))
else:
    print("   ✅ Error esperado al intentar descifrar mensaje no cifrado:")
    print(f"      Estado: {datos_descifrados_error.status}")
    print(f"      Código de error: {datos_descifrados_error.stderr.strip()}")


🔴 Luis intentando descifrar un mensaje NO CIFRADO (Ejemplo de Error)...
   ✅ Error esperado al intentar descifrar mensaje no cifrado:
      Estado: no data was provided
      Código de error: gpg: WARNING: unsafe permissions on homedir '/content/pgp_lab/luis'
gpg: no valid OpenPGP data found.
[GNUPG:] NODATA 1
[GNUPG:] NODATA 2
[GNUPG:] FAILURE decrypt 4294967295
gpg: decrypt_message failed: Unknown system error


En resumen, PGP es un sistema que permite enviar correos electrónicos seguros. Cada persona tiene una llave pública y una llave privada. El mensaje se cifra con la llave pública del destinatario y solo él puede descifrarlo con su llave privada. Además, el remitente firma el mensaje para demostrar que realmente lo envió. Con esto se garantiza confidencialidad, integridad y autenticidad.